<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/jihyun_llama_1b_%EC%88%98%EC%A0%951023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Settings

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install peft

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.3 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Add the padding token to the tokenizer
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
# If you are using the model for training, also resize the model embeddings to include the padding token
model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Embedding(128257, 2048)

3. Dataset load, 전처리

In [ ]:
from datasets import load_dataset
from bs4 import BeautifulSoup

# HTML 태그 제거 함수
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# 데이터셋 로드
dataset = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/YonseiAI/convert_train.csv',
    split='train'
)

# Body와 Title 필드에서 HTML 태그 제거
def remove_html_tags(examples):
    if 'Body' in examples:
        examples['Body'] = clean_html(examples['Body'])
    if 'Title' in examples:
        examples['Title'] = clean_html(examples['Title'])
    return examples

# 데이터셋의 각 예시에서 HTML 태그 제거
dataset = dataset.map(remove_html_tags)

# 학습 및 검증 데이터셋으로 나누기
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']


Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

<ipython-input-11-7e54774fd13f>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()
/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
/usr/lib/python3.10/html/parser.py:342: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that 

4. Tokenizing

In [ ]:
def preprocess_function(examples):
    # Body 텍스트를 모델의 입력으로 사용
    inputs = examples['Body']

    # Tags 리스트를 개별 태그로 변환하여 사용
    targets = [convert_tags_to_list(tags) for tags in examples['Tags']]

    # Body에 대한 토큰화 (input_ids, attention_mask 생성)
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

    # Tags에 대한 토큰화 (input_ids를 labels로 설정)
    labels = [tokenizer(tag, max_length=128, padding="max_length", truncation=True)['input_ids'] for tag in targets]

    # 모델이 사용할 필드 (input_ids와 labels) 추가
    model_inputs["labels"] = labels
    return model_inputs

# 전처리 함수를 사용하여 데이터셋을 토크나이즈
tokenized_datasets = train_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets[0])  # 첫 번째 샘플 확인

{'Id': 56154843, 'Title': 'How can I write Fibonacci with for...of?', 'Body': "** How can I write a program that produce Fibonacci with for...of in javascript**\nI've tried this, and it works well\nfunction createFibonacci(number) {\n    var i;\n    var fib = []; // Initialize array!\n\n    fib[0] = 0;\n    fib[1] = 1;\n    for (i = 2; i <= number; i++) {\n      // Next fibonacci number = previous + one before previous\n      // Translated to JavaScript:\n      fib[i] = fib[i - 2] + fib[i - 1];\n      console.log(fib[i]);\n    }\n}\n\ncreateFibonacci(8);\n\nBut I'm wondering if it is possible to write it with for..of, is there any way to do this?\n", 'Tags': '<javascript>', 'CreationDate': '2019-05-15 17:35:02', 'Y': 'LQ_CLOSE', 'Cleaned_tag ': 'javascript', 'Cleaned_tag': 'javascript', 'Array_Tag': "['javascript']", 'Filtered_Array_Tag': "['javascript']", 'Filtered_Tag': 'javascript', 'Concat_Text': "How can I write Fibonacci with for...of? <p>** How can I write a program that produce

6. Train

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# 학습 설정 (remove_unused_columns 추가)
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",           # 출력 디렉토리
    eval_strategy="epoch",            # 평가 전략 (에포크마다 평가)
    learning_rate=5e-5,               # 학습률
    per_device_train_batch_size=8,    # 학습 시 배치 크기
    per_device_eval_batch_size=8,     # 평가 시 배치 크기
    weight_decay=0.01,                # 가중치 감쇠
    save_total_limit=3,               # 저장할 체크포인트 최대 수
    num_train_epochs=3,               # 학습 에포크 수
    predict_with_generate=True,       # 학습 중 생성 작업 수행 여부
    fp16=True,                        # 16-bit floating point 사용 (성능 최적화)
    logging_dir='./logs',             # 로그 디렉토리
    logging_steps=10,                 # 로그 기록 간격 (10 스텝마다 기록)
    remove_unused_columns=False       # 사용하지 않는 열 자동 제거 안함
)

# Trainer 설정
trainer = Seq2SeqTrainer(
    model=model,                      # 미세 조정할 모델
    args=training_args,               # 학습 설정
    train_dataset=train_dataset,      # 학습 데이터셋
    eval_dataset=eval_dataset,        # 검증 데이터셋
    tokenizer=tokenizer,              # 토크나이저
)

# 학습 시작
trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y', 'Cleaned_tag ', 'Cleaned_tag', 'Array_Tag', 'Filtered_Array_Tag', 'Filtered_Tag', 'Concat_Text']

7. Model 학습이 끝난 모델 저장

In [ ]:
# 학습이 끝나면 모델과 토크나이저 저장
model.save_pretrained("./fine_tuned_llama")
tokenizer.save_pretrained("./fine_tuned_llama")


8. 모델 평가

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# CSV 데이터셋을 불러옴
df = pd.read_csv('/content/drive/My Drive/YonseiAI/convert_train.csv')

# Body와 Tags 열을 사용
df['Tags'] = df['Tags'].apply(lambda x: x.strip('<>').split('><'))

# 다중 레이블을 이진화
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Tags'])

# 학습/검증 데이터를 나눔
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df['Body'], y, test_size=0.2, random_state=42)


In [ ]:
from transformers import BartForSequenceClassification, BartTokenizer, Trainer, TrainingArguments
import torch

# 모델과 토크나이저 로드
model_id = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_id)
model = BartForSequenceClassification.from_pretrained(model_id, num_labels=y.shape[1])

# 데이터셋을 토크나이즈
def preprocess_function(examples):
    return tokenizer(examples, padding=True, truncation=True, max_length=512)

X_train_tokenized = X_train.apply(preprocess_function)
X_val_tokenized = X_val.apply(preprocess_function)

# Hugging Face의 DatasetDict 사용을 위해 데이터셋 준비
import datasets
train_dataset = datasets.Dataset.from_dict({'input_ids': X_train_tokenized, 'labels': y_train})
val_dataset = datasets.Dataset.from_dict({'input_ids': X_val_tokenized, 'labels': y_val})


In [ ]:
# 예측 및 샘플 출력
def predict_and_print_sample(sample_body, original_tags):
    # 본문 토크나이즈
    inputs = tokenizer(sample_body, return_tensors="pt", padding=True, truncation=True, max_length=512).to('cuda')

    # 모델 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_tags = torch.sigmoid(logits).cpu().numpy()

    # 예측된 태그를 바이너리화하고 원래 태그와 비교
    predicted_labels = (predicted_tags > 0.5).astype(int)
    predicted_tag_names = mlb.inverse_transform(predicted_labels)

    # 결과 출력
    print(f"Predicted Tags: {predicted_tag_names}")
    print(f"Original Tags: {original_tags}")

# 예측 및 결과 출력 (랜덤 샘플 5개)
for i in range(5):
    body = df['Body'].iloc[i]
    original_tags = df['Tags'].iloc[i]
    print(f"Sample {i + 1}:")
    predict_and_print_sample(body, original_tags)
